In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import poisson

In [5]:
data = pd.read_csv('accidents.csv', index_col=0)
data = data.iloc[:30]

In [51]:
def bootstrap_CI(X, m, alpha):
    # returns two-sided 
    n = len(X)
    X_resample = np.random.choice(X, size=(m,n))
    mles = np.mean(X_resample, axis=1)
    lower_bound = np.percentile(mles, alpha / 2)
    upper_bound = np.percentile(mles, 100 - alpha/2)
    return (lower_bound, upper_bound, mles)

In [63]:
np.random.seed(10)

l, u, mles = bootstrap_CI(data['values'], 10, 5)
print(l, u)

2.515 3.1958333333333337


In [62]:
from scipy.optimize import minimize_scalar

def poisson_log_likelihood(X, lam):
    likelihood = poisson.pmf(X, lam)
    return (-1) * np.sum(np.log(likelihood))

def bootstrap_CI2(X, m, alpha):
    # returns two-sided 
    n = len(X)
    X_resample = np.random.choice(X, size=(m,n))
    mles = np.zeros(m)
    for i in range(0, m):
        f = lambda lam: poisson_log_likelihood(X_resample[i,:], lam)
        res = minimize_scalar(f, bounds=(0, None))
        mles[i] = res.x
    lower_bound = np.percentile(mles, alpha / 2)
    upper_bound = np.percentile(mles, 100 - alpha/2)
    return (lower_bound, upper_bound, mles)

np.random.seed(10)

l1, u1, mles = bootstrap_CI2(data['values'], 10, 5)
print(l1, u1)

2.514999995753098 3.1958333407411548


<ipython-input-62-52d8f91ec63a>:5: RuntimeWarning: divide by zero encountered in log
  return (-1) * np.sum(np.log(likelihood))


In [60]:
from scipy.stats import gamma
from scipy.optimize import minimize


def gamma_log_likelihood(X, shape, scale):
    likelihood = gamma.pdf(X, shape, scale=scale)
    return (-1) * np.sum(np.log(likelihood))

def bootstrap_CI3(X, m, alpha):
    # returns two-sided 
    n = len(X)
    X_resample = np.random.choice(X, size=(m,n))
    mles = np.zeros((m,2))
    for i in range(0, m):
        f = lambda x: gamma_log_likelihood(X_resample[i,:], x[0], x[1])
        res = minimize(f, [1,1], bounds=((0, None), (0, None)))
        mles[i,:] = res.x
    lower_bound = np.percentile(mles, alpha / 2)
    upper_bound = np.percentile(mles, 100 - alpha/2)
    return (lower_bound, upper_bound, mles)

np.random.seed(10)

l1, u1, mles = bootstrap_CI3(data['values'] + 1, 10, 5)
print(l1, u1)

0.8901525501641778 4.567763582921698


<ipython-input-60-dedb880328ad>:7: RuntimeWarning: divide by zero encountered in log
  return (-1) * np.sum(np.log(likelihood))
/Users/christopherdolan/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Users/christopherdolan/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1835: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


In [61]:
gamma.fit(data['values']+1, floc=0)

(4.297352258068876, 0, 0.930805705417666)